# WaveformDipole

This notebook provides examples on how to use siriuspy.ramp subpackage in order to manipulate booster ramps and storage ring migration power supply waveforms. 

In [ ]:
from lnls import notebook
notebook.turn_code_on_off_html()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from siriuspy.ramp.waveform import WaveformDipole

## Bare constructor

In [ ]:
# With no arguments the constructor creates a default waveform for BO-Fam:MA-B:

w = WaveformDipole()

%matplotlib inline
plt.plot(w)
plt.xlabel('Index'); plt.ylabel('Waveform [GeV]')
plt.grid('on')
plt.show()

# By default the value 1.0 should correspond to the beam ejection point, 
# whereas the value 1.0/20 (20 = E_eje/E_inj) should correspond to the injection point:

idx_inj = 160
idx_eje = 2400
print('value at maximum         : {:.8f}'.format(max(w)))
print('value at injection point : {:.8f} (idx = {})'.format(w[idx_inj],idx_inj))
print('value at ejection point  : {:.8f} (idx = {})'.format(w[idx_eje],idx_eje))

## Parametrization

The parametrized component of the waveform is composed of 5 disctinct regions:

1) 0.0 <= t < rampup_start

2) rampup_start <= t < rampup_stop

3) rampup_stop <= t < rampdown_start

4) rampdown_start <= t < rampdown_stop

5) rampdown_stop <= t <= duration

In [ ]:
w = WaveformDipole()
print('Regions internal limits')
print('')
print('Region 1 (cubic) ,   {0:06.2f} ms <= t < {1:06.2f} ms'.format(0.0, w.rampup_start_time))
print('Region 2 (linear),   {0:06.2f} ms <= t < {1:06.2f} ms (ramp up)'.format(w.rampup_start_time, w.rampup_stop_time))
print('Region 3 (variable), {0:06.2f} ms <= t < {1:06.2f} ms (plateau)'.format(w.rampup_stop_time, w.rampdown_start_time))
print('Region 4 (linear),   {0:06.2f} ms <= t < {1:06.2f} ms (ramp down)'.format(w.rampdown_start_time, w.rampdown_stop_time))
print('Region 5 (cubic),    {0:06.2f} ms <= t < {1:06.2f} ms '.format(w.rampdown_stop_time, w.duration))

%matplotlib inline

# Region 1 (blue)
t = np.linspace(0, w.rampup_start_time, 50)
plt.plot(t, w.eval_at(t))

# Region 2 (yellow)
t = np.linspace(w.rampup_start_time, w.rampup_stop_time, 50)
plt.plot(t, w.eval_at(t))

# region 3 (green)
t = np.linspace(w.rampup_stop_time, w.rampdown_start_time, 50)
plt.plot(t, w.eval_at(t))

# Region 4 (red)
t = np.linspace(w.rampdown_start_time, w.rampdown_stop_time, 50)
plt.plot(t, w.eval_at(t))

# Region 5 (purple)
t = np.linspace(w.rampdown_stop_time, w.duration, 50)
plt.plot(t, w.eval_at(t))

plt.show()

## Changing RampUp and RampDown

There are utility methods that allow changing both linear ramp up and ramp down without waveform 'overshoots' and guaranteeing continuities. For that, the first region boundary and *plateau_start_index* are reset for ramp up changes, as collateral effect. Likewise, *plateau_stop_index* and the last region boundary are reset for ramp down changes:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from siriuspy.ramp.waveform import WaveformDipole

w = WaveformDipole()

# errors before change:
plt.plot(w.times, w.waveform)
print('errors list (before change): ', w.errors)

# changing rampup_stop_time
w.rampup_stop_time *= 0.91
plt.plot(w.times, w.waveform)
print('errors list (change rampup_stop_time): ', w.errors)

# changing rampdown_start_time
w.rampdown_start_time *= 1.01
plt.plot(w.times, w.waveform)
print('errors list (change rampdown_start_time): ', w.errors)


plt.legend()
plt.xlabel('Time [ms]'); plt.ylabel('Waveform [GeV]')
plt.grid('on')
plt.show()